# droitGPT
### AI Assistant specialized in French legal codes

# The language model
### Qwen-1_8B-Chat-Int4

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("backend/tokenizer/", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("backend/llm/", device_map="auto", trust_remote_code=True).eval()

user_q = ["Salut!", "Quelle est la capitale du Pérou?", "Qui est le président du Pérou?"]

history_in = None
for query in user_q:
    response, history_out = model.chat(tokenizer, query, history=history_in)
    print("User: ", query)
    print("Assistant: ", response)
    print("Chat history: ", history_out)
    history_in = history_out
    print()

/home/adrian/Desktop/MinesParistech2023-2024/NLP/droitGPT/backend/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


User:  Salut!
Assistant:  Bonjour ! Comment puis-je vous aider aujourd'hui ?
Chat history:  [('Salut!', "Bonjour ! Comment puis-je vous aider aujourd'hui ?")]

User:  Quelle est la capitale du Pérou?
Assistant:  La capital du Péninque est conjointement située au sud de la前台, dans le comté de Genève, aux préfectures de Couva et de Moroni.
Chat history:  [('Salut!', "Bonjour ! Comment puis-je vous aider aujourd'hui ?"), ('Quelle est la capitale du Pérou?', 'La capital du Péninque est conjointement située au sud de la前台, dans le comté de Genève, aux préfectures de Couva et de Moroni.')]

User:  Qui est le président du Pérou?
Assistant:  Le président du Péninque est actuellement Jean-Paul Bialetti.
Chat history:  [('Salut!', "Bonjour ! Comment puis-je vous aider aujourd'hui ?"), ('Quelle est la capitale du Pérou?', 'La capital du Péninque est conjointement située au sud de la前台, dans le comté de Genève, aux préfectures de Couva et de Moroni.'), ('Qui est le président du Pérou?', 'Le présid

# The Embeddings Model 
### Sentence Transformers
- Il faut trouver un modèle Sentence Transformers gratuit (donc pas de OpenAI Embeddings ni Ollama Embeddings)
- Ce modèle doit être pre-entrainé pour la comparaison semantique
- Ce modèle doit comprendre le français

In [2]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings_model_id = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_id)

/home/adrian/Desktop/MinesParistech2023-2024/NLP/droitGPT/backend/venv/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


# Vector Database
### FAISS

In [3]:
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document

docs = [Document(page_content="Le suffrage est direct et universel."),
        Document(page_content="Sont électeurs les Françaises et Français âgés de dix-huit ans accomplis, jouissant de leurs droits civils et politiques et n'étant dans aucun cas d'incapacité prévu par la loi."),
        Document(page_content="Ne doivent pas être inscrits sur la liste électorale, pendant le délai fixé par le jugement, ceux auxquels les tribunaux ont interdit le droit de vote et d'élection, par application des lois qui autorisent cette interdiction."),
        ]
vector = FAISS.from_documents(docs, embeddings)

# Querying the Database
On voit que les docs récuperés de la base de données sont un peu similaires au requête (modèle Embeddings n'est pas très grand)

In [4]:
def print_context(docs_and_scores):
    for i, (doc, score) in enumerate(docs_and_scores):
        print(f"Top {i+1}")
        print(doc.page_content)
        print()

In [5]:
query = "suffrage universel"
docs_and_scores = vector.similarity_search_with_score(query)
print_context(docs_and_scores)

Top 1
Le suffrage est direct et universel.

Top 2
Sont électeurs les Françaises et Français âgés de dix-huit ans accomplis, jouissant de leurs droits civils et politiques et n'étant dans aucun cas d'incapacité prévu par la loi.

Top 3
Ne doivent pas être inscrits sur la liste électorale, pendant le délai fixé par le jugement, ceux auxquels les tribunaux ont interdit le droit de vote et d'élection, par application des lois qui autorisent cette interdiction.



# Corpus

In [6]:
import os 
import re
from typing import List

def clean_data(data_folder: str, clean_data_folder: str, files_for_indexing: List[str]):
    if not os.path.exists(clean_data_folder):
        os.makedirs(clean_data_folder)

    for file in os.listdir(data_folder):
        if file in files_for_indexing:
            file_path = data_folder + "/" + file
            
            with open(file_path, "r", encoding="utf-8") as f:
                text = f.read()

                pattern1 = re.compile(r"---.*?---", re.DOTALL)  # remove title and date
                text = re.sub(pattern1, "", text)
                    
                pattern3 = re.compile(
                    r"<div.*</div>"
                )  # remove html content (simple approach, usually tables)
                text = re.sub(pattern3, "", text)

                pattern4 = re.compile(r"([:;])\n+")  # form paragraphs
                text = re.sub(pattern4, lambda x: x.group(1) + " ", text)

                add_context_to_article(text=text, clean_data_folder=clean_data_folder, file=file)

def add_context_to_article(
        clean_data_folder: str,
        text: str,
        file: str,
        init_level=2,
        header_name: str = None,
    ):
        level = init_level
        pattern = "#{level} (.*?)\n(.*?)(?=\n#{level} )"
        formatted_pattern = pattern.format(level="{" + str(level) + "}")
        parts = re.findall(formatted_pattern, text.strip(), re.DOTALL)
        if not parts:
            if ":" not in header_name:
                header_name = ""
            else:
                header_name = header_name.split(":")[-1].strip()

            articles = re.findall(r"\*\*(Art\. .*?)\*\*\n(.*?)(?=\*\*)", text, re.DOTALL)
            articles_context = [
                (article_name, "Contexte: " + header_name + "\n" + article_text)
                for article_name, article_text in articles
            ]
            for article_name, text in articles_context:
                text = text.strip()
                if not text:
                    continue
                clean_file_path = clean_data_folder + "/" + article_name + "_" + file
                with open(clean_file_path, "w", encoding="utf-8") as f:
                    f.write(text)

        for part_name, part_text in parts:
            add_context_to_article(
                text=part_text,
                header_name=part_name,
                init_level=level + 1,
                file=file,
            )
